## Orquestração de Tarefas com **Prefect**

![prefect](https://miro.medium.com/v2/resize:fit:1200/1*Mq0CqXzlHOJVysq1v_a5dA.png)



**Prefect** é uma poderosa ferramenta de orquestração de fluxos de trabalho moderna, focada na simplicidade e na robustez. Ela facilita a criação, agendamento e monitoramento de pipelines complexos de forma eficiente, com integração nativa a cron jobs, sistemas de tolerância a falhas e muito mais.



### O que é **Prefect**?

**Prefect** é uma plataforma open-source para a orquestração de tarefas em Python. Ele oferece uma maneira fácil de coordenar tarefas complexas com dependências e permite que você monitore o status das execuções. Prefect pode ser executado localmente ou na nuvem.

#### Principais características:
- Orquestração fácil de fluxos de trabalho.
- Sistema de agendamento flexível com suporte a cron jobs.
- Monitoramento detalhado de execuções e falhas.
- Integração com várias plataformas de dados (AWS, GCP, etc.).
- Fácil integração com Prefect Cloud para monitoramento centralizado.

---


#### Instalando o **Prefect**:

```bash
pip install prefect
```

#### Testando a Instalação:

Verifique se a instalação foi bem-sucedida:

```bash
prefect diagnostics
```

Isso exibirá detalhes sobre o ambiente e a instalação.

---

### Criando um Fluxo de Trabalho Básico

No Prefect, a unidade básica de execução é o **Task**, e um conjunto de tarefas é chamado de **Flow**.

#### Criando um fluxo básico:

```python
from prefect import task, Flow

# Definindo uma tarefa simples
@task
def saudacao():
    print("Olá! Esta é uma tarefa Prefect.")

# Criando um fluxo que executa a tarefa
with Flow("fluxo_basico") as flow:
    saudacao()

# Executando o fluxo
if __name__ == '__main__':
    flow.run()
```

- O **@task** é um decorador que transforma uma função Python em uma tarefa Prefect.
- O **Flow** agrupa uma ou mais tarefas que serão executadas juntas.
- O **flow.run()** executa o fluxo.

---


#### Exemplo com múltiplas tarefas:

```python
from prefect import task, Flow

# Tarefa 1
@task
def tarefa_1():
    print("Executando tarefa 1")

# Tarefa 2
@task
def tarefa_2():
    print("Executando tarefa 2")

# Criando o fluxo
with Flow("fluxo_com_multiplas_tarefas") as flow:
    t1 = tarefa_1()
    t2 = tarefa_2()

# Executando o fluxo
if __name__ == '__main__':
    flow.run()
```

Nesse exemplo, ambas as tarefas são executadas em paralelo, pois não há dependências entre elas.

---

### Adicionando Dependências entre Tarefas

Para criar dependências entre as tarefas, podemos usar a função `set_downstream` ou usar a sintaxe de operador `>>` (semelhante ao Airflow).


```python
from prefect import task, Flow

@task
def tarefa_1():
    print("Tarefa 1 executada")

@task
def tarefa_2():
    print("Tarefa 2 executada")

@task
def tarefa_3():
    print("Tarefa 3 executada")

# Criando o fluxo com dependências
with Flow("fluxo_com_dependencias") as flow:
    t1 = tarefa_1()
    t2 = tarefa_2()
    t3 = tarefa_3()

    # Definindo a ordem de execução
    t1.set_downstream(t2)
    t2.set_downstream(t3)

# Executando o fluxo
if __name__ == '__main__':
    flow.run()
```

#### Sintaxe alternativa com `>>`:

```python
with Flow("fluxo_com_dependencias_alternativo") as flow:
    t1 = tarefa_1()
    t2 = tarefa_2()
    t3 = tarefa_3()

    t1 >> t2 >> t3  # Definindo dependências em cascata
```

Neste exemplo, a **tarefa 2** só será executada após a conclusão da **tarefa 1**, e a **tarefa 3** só será executada após a **tarefa 2**.

---

### Agendamento com **Schedules**

Para agendar tarefas com diferentes frequências, **Prefect** oferece a classe `Schedule` e o `CronClock` para definir horários e intervalos de execução.

#### Exemplo com agendamento diário e horário:

```python
from prefect import task, Flow
from prefect.schedules import Schedule
from prefect.schedules.clocks import CronClock

@task
def tarefa_diaria():
    print("Tarefa executada uma vez ao dia")

@task
def tarefa_horaria():
    print("Tarefa executada a cada hora")

# Definindo um agendamento para rodar diariamente às 9h
schedule_diario = Schedule(clocks=[CronClock("0 9 * * *")])

# Definindo um agendamento para rodar no início de cada hora
schedule_horario = Schedule(clocks=[CronClock("0 * * * *")])

# Fluxo para a tarefa diária
with Flow("fluxo_diario", schedule=schedule_diario) as flow_diario:
    tarefa_diaria()

# Fluxo para a tarefa horária
with Flow("fluxo_horario", schedule=schedule_horario) as flow_horario:
    tarefa_horaria()

if __name__ == '__main__':
    flow_diario.run()
    flow_horario.run()
```


- Usamos `CronClock` para definir os horários de execução com base em expressões cron. No exemplo, a **tarefa diária** roda às 9h e a **tarefa horária** a cada hora.




#### Como usar `dbutils.notebook.run`?

O comando **`dbutils.notebook.run`** aceita três parâmetros principais:

- **notebook_path**: O caminho do notebook a ser executado.
- **timeout_seconds**: Tempo limite de execução (em segundos).
- **arguments**: Dicionário com parâmetros que podem ser passados para o notebook chamado.

#### Exemplo de uso básico:

```python
# Executando um notebook localizado em '/Users/meu_usuario/meu_notebook'
result = dbutils.notebook.run("/Users/meu_usuario/meu_notebook", 
                              timeout_seconds=300, 
                              arguments={"param1": "valor1"})

# Exibindo o resultado da execução do notebook
print(f"Resultado do notebook: {result}")
```

##### Como passar e acessar parâmetros:
Se o notebook chamado estiver esperando parâmetros, você pode usar o comando `dbutils.widgets.get()` para capturar esses argumentos.

**Notebook Chamado**:

```python
# Notebook que recebe parâmetros
param1 = dbutils.widgets.get("param1")
print(f"Recebido o parâmetro: {param1}")

# Retorna um valor para o notebook chamador
dbutils.notebook.exit(f"O parâmetro recebido foi: {param1}")
```

Neste exemplo, o notebook que está sendo chamado captura o valor `param1` e pode utilizá-lo em sua execução.

---

###  Integração do **Prefect** com `dbutils.notebook.run`


Aqui estão duas tasks que executam notebooks diferentes no Databricks. Uma recebe parâmetros e a outra depende do resultado da primeira.

```python
from prefect import task, Flow

# Task para executar um notebook no Databricks usando dbutils.notebook.run
@task
def executar_notebook_1():
    # Executa o primeiro notebook com parâmetros
    result = dbutils.notebook.run("/Users/seu_usuario/notebook_1", 
                                  timeout_seconds=300, 
                                  arguments={"param1": "valor1"})
    print(f"Resultado do Notebook 1: {result}")
    return result

# Task para executar um segundo notebook usando o resultado do primeiro
@task
def executar_notebook_2(param):
    # Executa o segundo notebook, passando o resultado do primeiro como parâmetro
    result = dbutils.notebook.run("/Users/seu_usuario/notebook_2", 
                                  timeout_seconds=300, 
                                  arguments={"param_from_notebook_1": param})
    print(f"Resultado do Notebook 2: {result}")
    return result
```


Com as tasks definidas, podemos agora criar um fluxo no **Prefect** que organiza a execução desses notebooks com dependências entre eles. Neste exemplo, o **notebook 2** será executado somente após o **notebook 1** e receberá o resultado do primeiro como parâmetro.

```python
from prefect import Flow

# Definir o fluxo com as tasks
with Flow("execucao_de_notebooks_databricks") as flow:
    # Executa o notebook 1
    resultado_notebook_1 = executar_notebook_1()
    
    # Passa o resultado do notebook 1 como argumento para o notebook 2
    executar_notebook_2(param=resultado_notebook_1)

# Executando o fluxo
if __name__ == "__main__":
    flow.run()
```

